# Helper Functions

In [39]:
def load_data(path, subject):
    # path where to save the datasets.
    bids_root = path
    subject_id = subject


    bids_path = BIDSPath(subject=subject_id,task="N170",session="N170",
                     datatype='eeg', suffix='eeg',
                     root=bids_root)

    # read the file
    raw = read_raw_bids(bids_path)
    # fix the annotations reading
    ccs_eeg_utils.read_annotations_core(bids_path,raw)

    raw.load_data()
    return raw

In [40]:
def add_channel_locations(raw):
    # tell compiler that these are no eeg channels
    raw.set_channel_types({'HEOG_left':'eog'})
    raw.set_channel_types({'HEOG_right':'eog'})
    raw.set_channel_types({'VEOG_lower':'eog'})

    raw.set_montage('standard_1020',match_case=False)

In [41]:
def filter_data(raw, low_bound, high_bound):
    # using mne filters for the eeg signal
    filtered_raw = raw.copy().filter(low_bound,high_bound, fir_design='firwin')
    return filtered_raw

In [42]:
def bad_segments(raw, subject_id):
    #find_bad_segments(raw, subject_id)
    load_bad_segments(raw, subject_id)
    
    if subject_id == '001':
        raw.info['bads'] = ['F8']

In [43]:
def find_bad_segments(raw, subject_id):
    # press a and mark bad channels by hand (holding right click)
    raw.plot(n_channels=len(raw.ch_names))
    # save bad channels
    #bad_ix = [i for i,a in enumerate(raw.annotations) if a['description']=="BAD_"]
    #raw.annotations[bad_ix].save("sub-{}_task-N170_badannotations.csv".format(subject_id))

In [44]:
def load_bad_segments(raw, subject_id):
    # load bad channels
    annotations = mne.read_annotations("sub-{}_task-N170_badannotations.csv".format(subject_id))
    raw.annotations.append(annotations.onset,annotations.duration,annotations.description)

In [45]:
def ica_filtering(raw, subject_id):
    # setup data with filter frequence 1.5Hz
    ica_raw = raw.copy()
    ica_raw.load_data().filter(l_freq=1, h_freq=None)
    ica = mne.preprocessing.ICA(method="fastica", random_state=111)
    ica.fit(raw,verbose=True)
    
    # max range = 30 - bad_channels (bad channels = 1 for Sub001)
    #ica.plot_components(range(30))
    # ica.plot_sources(raw, show_scrollbars=False)
    
    icaact = ica.get_sources(ica_raw)
    #plt.plot(icaact[5,0:20000][0].T)
    #plt.plot(ica_raw[0,0:20000][0].T)
    
    reload(mne)
    #ica.plot_properties(ica_raw,picks=[0,1],psd_args={'fmax': 35.},reject=None)
    
    # this is used to find bad components from ICA by hand
    #%matplotlib inline
    ica.plot_properties(inst=ica_raw, dB=False, topomap_args={'extrapolate': 'head', 'contours': 0},
                            psd_args={'fmin': 0, 'fmax': 50}, picks=['eeg'])
    
    eog_ind= []
    # find bad components via eog
    #eog_ind, _ = ica.find_bads_eog(raw)
    print('EOG Bad Components: ' + str(eog_ind))
    
    reconst_raw = raw.copy()
    # eog bads plus some additional found via analysing the spectrum of the components (likely muscle articats)
    expert_analysis_list = []
    if subject_id == '001':
        expert_analysis_list = [5,6,14,21]
    elif subject_id == '017':
        expert_analysis_list = [9]
    elif subject_id == '031':
        expert_analysis_list = []
        
    exclude_list = eog_ind
    for element in expert_analysis_list:
        if element not in exclude_list:
            exclude_list.append(element)
    ica.apply(reconst_raw,exclude=exclude_list)

    raw.plot()
    reconst_raw.plot()  
    ica.plot_overlay(raw,exclude=exclude_list)
    return reconst_raw

# Load Data

In [47]:
import osfclient
import mne
import mne_bids
import numpy as np
import ccs_eeg_utils
from ccs_eeg_semesterproject import (load_precomputed_badData, load_precomputed_ica)
from mne_bids import (BIDSPath,read_raw_bids)
from matplotlib import pyplot as plt
from importlib import reload  

%matplotlib qt
path = "../local/bidsN170"
temp_path = "/ses-N170/eeg/"
sub = '001'

# Preprocessing For Subjects 001, 017, 031

In [48]:
raw = load_data(path, sub)

# plot all channels (press a in that window)
# raw.plot(n_channels=len(raw.ch_names))#,scalings =40e-6)
# raw.plot_psd(area_mode='range', tmax=10.0, average=False,xscale="linear")

Reading ..\local\bidsN170\sub-001\ses-N170\eeg\sub-001_ses-N170_task-N170_eeg.fdt
Reading events from ..\local\bidsN170\sub-001\ses-N170\eeg\sub-001_ses-N170_task-N170_events.tsv.
Reading 0 ... 699391  =      0.000 ...   682.999 secs...


<ipython-input-39-8ec0acff4cc9>:12: RuntimeWarning: Data file name in EEG.data (sub-001_task-N170_eeg.fdt) is incorrect, the file name must have changed on disk, using the correct file name (sub-001_ses-N170_task-N170_eeg.fdt).
  raw = read_raw_bids(bids_path)
<ipython-input-39-8ec0acff4cc9>:12: RuntimeWarning: Expected to find a single channels file associated with sub-001_ses-N170_task-N170, but found 2: "['..\\local\\bidsN170\\sub-001\\ses-N170\\eeg\\sub-001_ses-N170_task-N170_badChannels.tsv', '..\\local\\bidsN170\\sub-001\\ses-N170\\eeg\\sub-001_ses-N170_task-N170_channels.tsv']".

The search_str was "..\local\bidsN170\sub-001\**\sub-001_ses-N170*channels.tsv"
  raw = read_raw_bids(bids_path)
<ipython-input-39-8ec0acff4cc9>:12: RuntimeWarning: Did not find any coordsystem.json associated with sub-001_ses-N170_task-N170.

The search_str was "..\local\bidsN170\sub-001\**\sub-001_ses-N170*coordsystem.json"
  raw = read_raw_bids(bids_path)
<ipython-input-39-8ec0acff4cc9>:12: RuntimeWa

In [19]:
raw.plot(n_channels=len(raw.ch_names))

Channels marked as bad: none


In [49]:
add_channel_locations(raw)

In [50]:
raw.plot_psd(fmax=75., average=True, spatial_colors=False)
# plt.savefig('./preprocessing_images/testtest')

Effective window size : 2.000 (s)


In [51]:
raw_filtered = filter_data(raw, 0.5, 50)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 6759 samples (6.601 sec)



In [52]:
# as sanity check
raw_filtered.plot_psd(fmax=75., average=True, spatial_colors=False)
# plt.savefig('./preprocessing_images/testtest')

Effective window size : 2.000 (s)


In [53]:
raw = raw_filtered

In [54]:
# To detect waves of x Hz you need at least sampling with 2*x Hz (Nyquistâ€“Shannon sampling theorem)
# raw_resampled = raw.copy().resample(100, npad='auto')  

In [55]:
bad_segments(raw, sub)

In [56]:
raw_ica = ica_filtering(raw, sub)

Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 3381 samples (3.302 sec)

Fitting ICA to data using 29 channels (please be patient, this may take a while)
Omitting 20361 of 699392 (2.91%) samples, retaining 679031 (97.09%) samples.
Selecting by non-zero PCA components: 29 components
Fitting ICA took 27.7s.
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
Not setting metadata
324 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
Not setting metadata
Not setting metadata
324 matching events found
No baseline correction applied
0 proj

C:\Users\Kay\anaconda3\lib\site-packages\mne\viz\ica.py:110: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=figsize, facecolor=[0.95] * 3)


Not setting metadata
Not setting metadata
324 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
Not setting metadata
Not setting metadata
324 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
Not setting metadata
Not setting metadata
324 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
Not setting metadata
Not setting metadata
324 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
Not setting metadata
Not setting metadata
324 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
Not setting metadata
Not setting metadata
324 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
Not setting metadata
Not setting metadata
324 matching events found
No baseline correction applied
0 proje

In [82]:
raw_ica.interpolate_bads()

<ipython-input-82-22e9af2ea844>:1: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  raw_ica.interpolate_bads()


<RawEEGLAB | sub-031_ses-N170_task-N170_eeg.fdt, 33 x 576512 (563.0 s), ~145.2 MB, data loaded>

In [83]:
raw = raw_ica

In [84]:
raw_rereference, _  = mne.set_eeg_reference(raw, 'average', projection=True)
raw_rereference.apply_proj()

Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...


<RawEEGLAB | sub-031_ses-N170_task-N170_eeg.fdt, 33 x 576512 (563.0 s), ~145.2 MB, data loaded>

In [85]:
save_path = path + "/sub-" + sub + temp_path + "sub-" + sub +"_cleaned.fif"
raw_rereference.save(save_path, overwrite=True)

Overwriting existing file.
Writing C:\Users\Kay\Desktop\Master4\BRAIN\local\bidsN170\sub-031\ses-N170\eeg\sub-031_cleaned.fif
Closing C:\Users\Kay\Desktop\Master4\BRAIN\local\bidsN170\sub-031\ses-N170\eeg\sub-031_cleaned.fif


<ipython-input-85-90e75c1d979a>:2: RuntimeWarning: This filename (C:\Users\Kay\Desktop\Master4\BRAIN\local\bidsN170\sub-031\ses-N170\eeg\sub-031_cleaned.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  raw_rereference.save(save_path, overwrite=True)


[done]


# Preprocessing For The Remaining Subjects

In [13]:
for s in range(1,41):
    # if we preprocessed the subject already, go on with next subject
    if s in [1,17,31]:
        continue
    if len(str(s)) > 1:
        sub = '0' + str(s)
    else:
        sub = '00' + str(s)
    raw = load_data(path, sub)
    add_channel_locations(raw)
    raw = filter_data(raw)
    
    # load preprocessing information
    annotations, bad_ch = load_precomputed_badData(bids_root=path, subject_id=sub, task="N170")
    ica, bad_components = load_precomputed_ica(bids_root=path, subject_id=sub, task="N170")
    
    ica.apply(raw, exclude=bad_components)
    
    raw_rereference, _  = mne.set_eeg_reference(raw, 'average', projection=True)
    raw_rereference.apply_proj()
    
    save_path = path + "/sub-" + sub + temp_path + "sub-" + sub +"_cleaned.fif"
    raw_rereference.save(save_path, overwrite=True)
print('DONE')

Reading ..\local\bidsN170\sub-002\ses-N170\eeg\sub-002_ses-N170_task-N170_eeg.fdt
Reading events from ..\local\bidsN170\sub-002\ses-N170\eeg\sub-002_ses-N170_task-N170_events.tsv.
Reading 0 ... 683007  =      0.000 ...   666.999 secs...


<ipython-input-2-8ec0acff4cc9>:12: RuntimeWarning: Data file name in EEG.data (sub-002_task-N170_eeg.fdt) is incorrect, the file name must have changed on disk, using the correct file name (sub-002_ses-N170_task-N170_eeg.fdt).
  raw = read_raw_bids(bids_path)
<ipython-input-2-8ec0acff4cc9>:12: RuntimeWarning: Expected to find a single channels file associated with sub-002_ses-N170_task-N170, but found 2: "['..\\local\\bidsN170\\sub-002\\ses-N170\\eeg\\sub-002_ses-N170_task-N170_badChannels.tsv', '..\\local\\bidsN170\\sub-002\\ses-N170\\eeg\\sub-002_ses-N170_task-N170_channels.tsv']".

The search_str was "..\local\bidsN170\sub-002\**\sub-002_ses-N170*channels.tsv"
  raw = read_raw_bids(bids_path)
<ipython-input-2-8ec0acff4cc9>:12: RuntimeWarning: Did not find any coordsystem.json associated with sub-002_ses-N170_task-N170.

The search_str was "..\local\bidsN170\sub-002\**\sub-002_ses-N170*coordsystem.json"
  raw = read_raw_bids(bids_path)
<ipython-input-2-8ec0acff4cc9>:12: RuntimeWarnin

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 6759 samples (6.601 sec)

..\local\bidsN170\sub-002\ses-N170\eeg\sub-002_ses-N170_task-N170_
Applying ICA to Raw instance


C:\Users\Kay\Desktop\Master4\BRAIN\Semesterproject\ccs_eeg_semesterproject.py:59: UserWarning: loadtxt: Empty input file: "..\local\bidsN170\sub-002\ses-N170\eeg\sub-002_ses-N170_task-N170_badChannels.tsv"
  badChannels = np.loadtxt(fn+'badChannels.tsv',delimiter='\t')


    Transforming to ICA space (30 components)
    Zeroing out 2 ICA components
    Projecting back using 30 PCA components
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...
Overwriting existing file.
Writing C:\Users\Kay\Desktop\Master4\BRAIN\local\bidsN170\sub-002\ses-N170\eeg\sub-002_cleaned.fif


...

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 6759 samples (6.601 sec)

..\local\bidsN170\sub-040\ses-N170\eeg\sub-040_ses-N170_task-N170_


C:\Users\Kay\Desktop\Master4\BRAIN\Semesterproject\ccs_eeg_semesterproject.py:59: UserWarning: loadtxt: Empty input file: "..\local\bidsN170\sub-040\ses-N170\eeg\sub-040_ses-N170_task-N170_badChannels.tsv"
  badChannels = np.loadtxt(fn+'badChannels.tsv',delimiter='\t')


Applying ICA to Raw instance
    Transforming to ICA space (30 components)
    Zeroing out 3 ICA components
    Projecting back using 30 PCA components
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...
Overwriting existing file.
Writing C:\Users\Kay\Desktop\Master4\BRAIN\local\bidsN170\sub-040\ses-N170\eeg\sub-040_cleaned.fif


<ipython-input-13-5dd91fd5c82e>:23: RuntimeWarning: This filename (C:\Users\Kay\Desktop\Master4\BRAIN\local\bidsN170\sub-040\ses-N170\eeg\sub-040_cleaned.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  raw_rereference.save(save_path, overwrite=True)


Closing C:\Users\Kay\Desktop\Master4\BRAIN\local\bidsN170\sub-040\ses-N170\eeg\sub-040_cleaned.fif
[done]
DONE


## save preprocessing without setting channel positions to avoide a strange bug in decoding

In [12]:
for s in range(1,41):
    # if we preprocessed the subject already, go on with next subject
    if s in [1,17,31]:
        continue
    if len(str(s)) > 1:
        sub = '0' + str(s)
    else:
        sub = '00' + str(s)
    raw = load_data(path, sub)
    raw = filter_data(raw)
    
    # load preprocessing information
    annotations, bad_ch = load_precomputed_badData(bids_root=path, subject_id=sub, task="N170")
    ica, bad_components = load_precomputed_ica(bids_root=path, subject_id=sub, task="N170")
    
    ica.apply(raw, exclude=bad_components)
    
    raw_rereference, _  = mne.set_eeg_reference(raw, 'average', projection=True)
    raw_rereference.apply_proj()
    
    save_path = path + "/sub-" + sub + temp_path + "sub-" + sub +"_cleaned_no_channel_positions.fif"
    raw_rereference.save(save_path, overwrite=True)
print('DONE')

Reading ..\local\bidsN170\sub-002\ses-N170\eeg\sub-002_ses-N170_task-N170_eeg.fdt
Reading events from ..\local\bidsN170\sub-002\ses-N170\eeg\sub-002_ses-N170_task-N170_events.tsv.
Reading 0 ... 683007  =      0.000 ...   666.999 secs...


<ipython-input-1-8ec0acff4cc9>:12: RuntimeWarning: Data file name in EEG.data (sub-002_task-N170_eeg.fdt) is incorrect, the file name must have changed on disk, using the correct file name (sub-002_ses-N170_task-N170_eeg.fdt).
  raw = read_raw_bids(bids_path)
<ipython-input-1-8ec0acff4cc9>:12: RuntimeWarning: Expected to find a single channels file associated with sub-002_ses-N170_task-N170, but found 2: "['..\\local\\bidsN170\\sub-002\\ses-N170\\eeg\\sub-002_ses-N170_task-N170_badChannels.tsv', '..\\local\\bidsN170\\sub-002\\ses-N170\\eeg\\sub-002_ses-N170_task-N170_channels.tsv']".

The search_str was "..\local\bidsN170\sub-002\**\sub-002_ses-N170*channels.tsv"
  raw = read_raw_bids(bids_path)
<ipython-input-1-8ec0acff4cc9>:12: RuntimeWarning: Did not find any coordsystem.json associated with sub-002_ses-N170_task-N170.

The search_str was "..\local\bidsN170\sub-002\**\sub-002_ses-N170*coordsystem.json"
  raw = read_raw_bids(bids_path)
<ipython-input-1-8ec0acff4cc9>:12: RuntimeWarnin

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 6759 samples (6.601 sec)

..\local\bidsN170\sub-002\ses-N170\eeg\sub-002_ses-N170_task-N170_
Applying ICA to Raw instance


C:\Users\Kay\Desktop\Master4\BRAIN\Semesterproject\ccs_eeg_semesterproject.py:59: UserWarning: loadtxt: Empty input file: "..\local\bidsN170\sub-002\ses-N170\eeg\sub-002_ses-N170_task-N170_badChannels.tsv"
  badChannels = np.loadtxt(fn+'badChannels.tsv',delimiter='\t')


    Transforming to ICA space (30 components)
    Zeroing out 2 ICA components
    Projecting back using 30 PCA components
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...
Writing C:\Users\Kay\Desktop\Master4\BRAIN\local\bidsN170\sub-002\ses-N170\eeg\sub-002_cleaned_no_channel_positions.fif


...

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 6759 samples (6.601 sec)

..\local\bidsN170\sub-040\ses-N170\eeg\sub-040_ses-N170_task-N170_


C:\Users\Kay\Desktop\Master4\BRAIN\Semesterproject\ccs_eeg_semesterproject.py:59: UserWarning: loadtxt: Empty input file: "..\local\bidsN170\sub-040\ses-N170\eeg\sub-040_ses-N170_task-N170_badChannels.tsv"
  badChannels = np.loadtxt(fn+'badChannels.tsv',delimiter='\t')


Applying ICA to Raw instance
    Transforming to ICA space (30 components)
    Zeroing out 3 ICA components
    Projecting back using 30 PCA components
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...
Writing C:\Users\Kay\Desktop\Master4\BRAIN\local\bidsN170\sub-040\ses-N170\eeg\sub-040_cleaned_no_channel_positions.fif


<ipython-input-12-e84577ec80fc>:22: RuntimeWarning: This filename (C:\Users\Kay\Desktop\Master4\BRAIN\local\bidsN170\sub-040\ses-N170\eeg\sub-040_cleaned_no_channel_positions.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  raw_rereference.save(save_path, overwrite=True)


Closing C:\Users\Kay\Desktop\Master4\BRAIN\local\bidsN170\sub-040\ses-N170\eeg\sub-040_cleaned_no_channel_positions.fif
[done]
DONE
